In [1]:
import numpy as np
import pandas as pd
import os
import itertools as it
import time

In [2]:
pd.options.display.max_rows = 500

In [3]:
# file = ''
# for i in os.listdir():
#     if 'NBA' in i:
#         file = i

In [4]:
raw = pd.read_csv('FanDuel-NBA-2019-04-07-34300-players-list.csv')

In [5]:
# raw

In [6]:
raw.columns

Index(['Id', 'Position', 'First Name', 'Nickname', 'Last Name', 'FPPG',
       'Played', 'Salary', 'Game', 'Team', 'Opponent', 'Injury Indicator',
       'Injury Details', 'Tier', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')

In [7]:
df = raw.drop(['Id', 'First Name', 'Last Name', 'Played', 'Game', 'Injury Details', 'Unnamed: 14', 'Unnamed: 15', 'Tier'], axis=1)

In [8]:
df = df[(df['Injury Indicator'] != 'O')]
df = df[(df['Injury Indicator'] != 'GTD')]
df.drop('Injury Indicator', axis=1, inplace=True)
df.Position.unique()

array(['SG', 'C', 'PF', 'PG', 'SF'], dtype=object)

In [9]:
df

,Position,Nickname,FPPG,Salary,Team,Opponent
1,SG,James Harden,58.222079,11800,HOU,PHO
3,C,Rudy Gobert,42.040506,10600,UTA,LAL
4,SG,Donovan Mitchell,37.418422,9700,UTA,LAL
5,C,Nikola Vucevic,44.913751,9700,ORL,BOS
6,PF,Julius Randle,37.702740,9200,NO,SAC
7,PG,Damian Lillard,43.555695,9200,POR,DEN
8,C,DeMarcus Cousins,37.382141,9100,GS,LAC
9,PG,Stephen Curry,44.263234,9100,GS,LAC
11,SF,Kevin Durant,45.324675,9000,GS,LAC
12,PG,Kyrie Irving,43.728359,8900,BOS,ORL


In [10]:
pg = df[df.Position == 'PG']
sg = df[df.Position == 'SG']
sf = df[df.Position == 'SF']
c = df[df.Position == 'C']
pf = df[df.Position == 'PF']

In [11]:
# def position(df):
#     pos = df.assing(foo=1).merge(df.assign(foo=1), on='foo').drop('foo', 1)
#     pos = pos[pos.Nickname_x != pos.Nickname_y]

### PG's

In [12]:
pgs = pg.assign(foo=1).merge(pg.assign(foo=1), on='foo').drop('foo', 1)
pgs.shape

(729, 12)

In [13]:
pgs = pgs[pgs.Nickname_x != pgs.Nickname_y]

In [14]:
pgs['ffpg'] = pgs.FPPG_x + pgs.FPPG_y
pgs['salary'] = pgs.Salary_x + pgs.Salary_y
pgs_no_opps = pgs[pgs.Team_x != pgs.Opponent_y]
pgs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [15]:
pgs.shape

(702, 4)

### SG's

In [16]:
sg.shape

(45, 6)

In [17]:
sgs = sg.assign(foo=1).merge(sg.assign(foo=1), on='foo').drop('foo', 1)
sgs = sgs[sgs.Nickname_x != sgs.Nickname_y]
sgs['ffpg'] = sgs.FPPG_x + sgs.FPPG_y
sgs['salary'] = sgs.Salary_x + sgs.Salary_y
sgs_no_opps = sgs[sgs.Team_x != sgs.Opponent_y]
sgs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [18]:
sgs.shape

(1980, 4)

### SF's

In [19]:
sf.shape

(46, 6)

In [20]:
sfs = sf.assign(foo=1).merge(sf.assign(foo=1), on='foo').drop('foo', 1)
sfs = sfs[sfs.Nickname_x != sfs.Nickname_y]
sfs['ffpg'] = sfs.FPPG_x + sfs.FPPG_y
sfs['salary'] = sfs.Salary_x + sfs.Salary_y
sfs_no_opps = sfs[sfs.Team_x != sfs.Opponent_y]
sfs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [21]:
sfs.shape

(2070, 4)

### PF's

In [22]:
pf.shape

(38, 6)

In [23]:
pfs = pf.assign(foo=1).merge(pf.assign(foo=1), on='foo').drop('foo', 1)
pfs = pfs[pfs.Nickname_x != pfs.Nickname_y]
pfs['ffpg'] = pfs.FPPG_x + pfs.FPPG_y
pfs['salary'] = pfs.Salary_x + pfs.Salary_y
pfs_no_opps = pfs[pfs.Team_x != pfs.Opponent_y]
pfs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [24]:
pfs.shape

(1406, 4)

In [25]:
# pfs = pfs[(pfs.Nickname_x != 'Anthony Davis') & (pfs.Nickname_y != 'Anthony Davis')]

### Forwards

In [26]:
forwards = pfs.assign(foo=1).merge(sfs.assign(foo=1), on='foo').drop('foo', 1)
forwards['ffpg'] = forwards.ffpg_x + forwards.ffpg_y
forwards['salary'] = forwards.salary_x + forwards.salary_y
forwards.drop(['ffpg_x', 'salary_x', 'ffpg_y', 'salary_y'], axis=1, inplace=True)

In [27]:
forwards.head()

,Nickname_x_x,Nickname_y_x,Nickname_x_y,Nickname_y_y,ffpg,salary
0,Julius Randle,Josh Jackson,Kevin Durant,Mario Hezonja,123.247383,33200
1,Julius Randle,Josh Jackson,Kevin Durant,Joe Ingles,133.288094,32900
2,Julius Randle,Josh Jackson,Kevin Durant,Jonathan Isaac,128.930925,31700
3,Julius Randle,Josh Jackson,Kevin Durant,Gordon Hayward,129.681212,31600
4,Julius Randle,Josh Jackson,Kevin Durant,Mikal Bridges,126.297919,31600


In [28]:
forwards.shape

(2910420, 6)

In [29]:
# forwards = forwards.iloc[:500000, :]

### Guards

In [34]:
guards = sgs.assign(foo=1).merge(pgs.assign(foo=1), on='foo').drop('foo', 1)
guards['ffpg'] = guards.ffpg_x + guards.ffpg_y
guards['salary'] = guards.salary_x + guards.salary_y
guards.drop(['ffpg_x', 'salary_x', 'ffpg_y', 'salary_y'], axis=1, inplace=True)

In [35]:
guards.columns = ['sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
guards.shape

(1389960, 6)

In [36]:
# guards = guards.iloc[:1000000,:]

In [37]:
guards = guards.sort_values(by='points', ascending=False)

In [38]:
gs = guards.iloc[::4, :]

In [39]:
guards.head()

,sg1,sg2,pg1,pg2,points,salary
61803,Bradley Beal,James Harden,Stephen Curry,Kyrie Irving,189.951944,38500
729,James Harden,Bradley Beal,Stephen Curry,Kyrie Irving,189.951944,38500
755,James Harden,Bradley Beal,Kyrie Irving,Stephen Curry,189.951944,38500
61829,Bradley Beal,James Harden,Kyrie Irving,Stephen Curry,189.951944,38500
61802,Bradley Beal,James Harden,Stephen Curry,Damian Lillard,189.779280,38800


In [40]:
gs.shape

(347490, 6)

In [41]:
# gs = gs[(gs.sg1 != 'Reggie Bullock') & (gs.sg2 != 'Reggie Bullock')]

### Centers and Forwards

In [42]:
cfs = c.assign(foo=1).merge(forwards.assign(foo=1), on='foo').drop('foo', 1)
cfs['ffpg'] = cfs.FPPG + cfs.ffpg
cfs['salary'] = cfs.Salary + cfs.salary
cfs.drop(['Position', 'FPPG', 'Salary', 'Team', 'Opponent'], axis=1, inplace=True)

In [43]:
cfs.columns = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'points', 'salary']
cfs.shape

(87312600, 7)

In [44]:
cfs = cfs.sort_values(by=['points'], ascending=False)

In [45]:
cf = cfs.iloc[::4, :]

In [46]:
cf.head(2)

,c,pf1,pf2,sf1,sf2,points,salary
2912580,Nikola Vucevic,Julius Randle,Aaron Gordon,Joe Ingles,Kevin Durant,188.020882,42300
3140280,Nikola Vucevic,Montrezl Harrell,Julius Randle,Joe Ingles,Kevin Durant,187.845092,42100


In [47]:
cf.shape

(21828150, 7)

### Full Line

In [ ]:
# cols = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
# df = pd.DataFrame(columns=cols)

In [64]:
# keep top 100, if < min of top 100, remove from gs
# take top 100 from line and concat
cols = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
df = pd.DataFrame(columns=cols)
t = time.time()
# gg = gs
for i in range(100000, 140000):
    gg = gs
    if df.shape[0] >= 100:
        df = df.sort_values(by=['points'], ascending=False)
        df = df.iloc[:100,:]
        gg =  gg[cf.iloc[i,5]+gg.points > df.points.min()]
       
    cap = gg[cf.iloc[i,6]+gg.salary<=60000]
    if cap.shape[0] > 0:
        cap = cap.sort_values(by=['points'], ascending=False)
        cap = cap.iloc[:100,:]
    else:
        continue
    r = cf.iloc[i,:].to_frame().transpose()
    row = pd.DataFrame(r).assign(foo=1).merge(cap.assign(foo=1), on='foo').drop('foo', 1)
    row['points'] = row.points_x + row.points_y
    row['salary'] = row.salary_x + row.salary_y
    row.drop(['points_x', 'points_y', 'salary_x', 'salary_y'], axis=1, inplace=True)
    df = df.append(row, ignore_index=True)
    if i>0 and i%1000==0:
        print(i)
print('time:', time.time() - t)
df


100000
time: 267.4393615722656


,c,pf1,pf2,sf1,sf2,sg1,sg2,pg1,pg2,points,salary
0,DeAndre Jordan,Marcus Morris,Kenneth Faried,Kevin Durant,Gerald Green,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,310.13,59900
1,Willie Cauley-Stein,Trey Lyles,Marcus Morris,Kevin Durant,Nemanja Bjelica,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,310.024,59800
2,DeAndre Jordan,Kenneth Faried,Marcus Morris,Jake Layman,Kevin Durant,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,309.747,59900
3,DeAndre Jordan,Kenneth Faried,Trey Lyles,Kevin Durant,Nemanja Bjelica,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,309.605,59600
4,Willie Cauley-Stein,Kenneth Faried,Trey Lyles,Harrison Barnes,Kevin Durant,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,309.53,60000
5,Willie Cauley-Stein,Marcus Morris,Trey Lyles,Gary Harris,Kevin Durant,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,309.455,60000
6,DeAndre Jordan,Kenneth Faried,Trey Lyles,Kevin Durant,Nemanja Bjelica,James Harden,Kent Bazemore,Damian Lillard,Stephen Curry,309.432,59900
7,DeAndre Jordan,Marcus Morris,Kenneth Faried,Kevin Durant,Gerald Green,James Harden,Kent Bazemore,Kyrie Irving,Damian Lillard,309.423,60000
8,DeAndre Jordan,Trey Lyles,PJ Tucker,Nemanja Bjelica,Kevin Durant,Kent Bazemore,James Harden,Kyrie Irving,Stephen Curry,309.323,59800
9,Willie Cauley-Stein,Trey Lyles,Marcus Morris,Kevin Durant,Nemanja Bjelica,James Harden,Kent Bazemore,Kyrie Irving,Damian Lillard,309.316,59900


In [67]:
# keep top 100, if < min of top 100, remove from gs
# take top 100 from line and concat
# cols = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
# df = pd.DataFrame(columns=cols)
# t = time.time()
# # gg = gs
# for i in range(10):
#     cc = cf
#     if df.shape[0] >= 100:
#         df = df.sort_values(by=['points'], ascending=False)
#         df = df.iloc[:100,:]
#         cc =  cc[gs.iloc[i,5]+cc.points > df.points.min()]
       
#     cap = cc[cf.iloc[i,6]+cc.salary<=60000]
#     if cap.shape[0] > 0:
#         cap = cap.sort_values(by=['points'], ascending=False)
#         cap = cap.iloc[:100,:]
#     else:
#         continue
#     r = gs.iloc[i,:].to_frame().transpose()
#     row = pd.DataFrame(r).assign(foo=1).merge(cap.assign(foo=1), on='foo').drop('foo', 1)
#     row['points'] = row.points_x + row.points_y
#     row['salary'] = row.salary_x + row.salary_y
#     row.drop(['points_x', 'points_y', 'salary_x', 'salary_y'], axis=1, inplace=True)
#     df = df.append(row, ignore_index=True)
#     if i>0 and i%1000==0:
#         print(i)
# print('time:', time.time() - t)
# df

In [ ]:
# set(gs.iloc[1, 0:4].values.tolist()) == set(gs.iloc[2,0:4].values.tolist())

# dups = []
# for i in range(gs.shape[0]-1):
#     check = set(gs.iloc[i, 0:4].values.tolist()) == set(gs.iloc[i+1,0:4].values.tolist())
#     dups.append(check)
# dups = dups + [False]



In [ ]:
# try guards vs cfs instead

In [63]:
df.to_csv('top100.csv')